<a href="https://colab.research.google.com/github/lazyboneOwO/PL-Repo/blob/main/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
                gradio pandas beautifulsoup4 google-generativeai python-dateutil
import os, time, uuid, re, json, datetime, tempfile
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

import gspread
from google.auth import default
try:
    #Colab 認證
    auth.authenticate_user()
    credits, _ = default()
    gc = gspread.authorize(credits)
    print("Google Sheets認證成功。")
except Exception as e:
    print(f"Google Sheets認證失敗:{e}。請在Colab環境中執行。")
    gc = None

from google.colab import userdata

SHEET_URL = "https://docs.google.com/spreadsheets/d/1MOVe4hGsyHXLZ-tCznSzHIUN5ppk6SC_wUEKQZQUpuE/edit?usp=sharing"
TIMEZONE = "Asia/Taipei"

# 任務表頭(Eng)
TASKS_HEADER_PROG = [
    "id","task","status","priority","est_min","actual_min","pomodoros",
    "completed_period","completed_date","notes", "due_date", "labels", "created_at",
    "updated_at", "planned_for", "start_time", "end_time"
]

# 任務表頭(Cn))
TASKS_COL_MAP = {
    "任務":"task", "狀態":"status", "優先級":"priority", "預估時間":"est_min",
    "實際完成時間":"actual_min", "番茄鐘數":"pomodoros", "完成時段":"completed_period",
    "完成日期":"completed_date", "備註":"notes"
}

# tasks 和 ai_chat 分頁
TASKS_SHEET_NAME = "tasks"
AI_SHEET_NAME = "ai_chat"
AI_HEADER = ["時間","任務列表","AI建議"]

def ensure_worksheet(sh, title, header_map=None):
    """確保 Google Sheet 存在指定分頁，並檢查/更新表頭。"""
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        if header_map is None:
             # For AI Chat
            expected_header_display = AI_HEADER
        else:
             # For Tasks
            expected_header_display = list(header_map.keys()) + ["id", "due_date", "labels", "created_at", "updated_at", "planned_for", "start_time", "end_time"]

        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(expected_header_display)+5))

    # 檢查並更新表頭
    if header_map:
        expected_header_display = list(header_map.keys()) + ["id", "due_date", "labels", "created_at", "updated_at", "planned_for", "start_time", "end_time"]
    else:
        expected_header_display = AI_HEADER

    data = ws.get_all_values()
    if not data or data[0][:len(expected_header_display)] != expected_header_display:
        ws.clear()
        ws.update([expected_header_display])
    return ws

try:
    gsheets = gc.open_by_url(SHEET_URL)
    print("成功開啟 Google試算表")
except Exception as e:
    print(f"開啟試算表失敗，請檢查 SHEET_URL或認證步驟：{e}")


# 確保分頁存在
ws_tasks = ensure_worksheet(gsheets, TASKS_SHEET_NAME, TASKS_COL_MAP)
ws_ai = ensure_worksheet(gsheets, AI_SHEET_NAME)

# Gemini API
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# 檢查是否載入
api_key_prefix = os.environ["GEMINI_API_KEY"][:5] if os.environ.get("GEMINI_API_KEY") else ""

if api_key_prefix:
    print(f"Gemini API Key載入成功(Prefix: {api_key_prefix}...)")
else:
    print("請確認您已在Colab中設置'GOOGLE_API_KEY'並成功載入")

Google Sheets認證成功。
成功開啟 Google試算表
Gemini API Key載入成功(Prefix: AIzaS...)


In [32]:
def tznow():
    """獲取當前時間"""
    return dt.now(gettz(TIMEZONE))

def read_data(ws, header_prog, col_map=None):
    """從 Google Sheet讀取資料並轉換為 List/Dicts(Eng)"""
    data = ws.get_all_records()
    if not data:
        return []

    processed_data = []
    prog_map = {k:v for k,v in col_map.items()} if col_map else {}

    for row in data:
        new_row = {}
        for k, v in row.items():
            if k in prog_map:
                new_row[prog_map[k]] = v
            else:
                new_row[k] = v

        for col in ["est_min", "actual_min", "pomodoros"]:
            try:
                new_row[col] = int(new_row.get(col) or 0)
            except ValueError:
                new_row[col] = 0

        if not new_row.get("id"):
             new_row["id"] = str(uuid.uuid4())[:8]

        processed_data.append(new_row)

    return processed_data

def write_data(ws, data, col_map):
    """將 List/Dicts寫回 Google Sheet(Cn)"""
    ws.clear()

    extra_keys = ["id", "due_date", "labels", "created_at", "updated_at", "planned_for", "start_time", "end_time"]

    if col_map:
        header_order = [k for k in col_map.keys()] + extra_keys
    else:
        header_order = AI_HEADER

    output_list = [header_order]

    if not data:
        ws.update([header_order])
        return

    for row_dict in data:
        row_list = []
        for header in header_order:
            prog_key = col_map.get(header, header) if col_map and header in col_map.keys() else header
            value = str(row_dict.get(prog_key, ""))
            row_list.append(value)
        output_list.append(row_list)

    ws.update(output_list)


def refresh_all():
    """從Google Sheet讀取資料到全域變數"""
    global tasks_data, clips_df

    tasks_data = read_data(ws_tasks, TASKS_HEADER_PROG, TASKS_COL_MAP)

    clips_df_raw = get_as_dataframe(ws_ai, evaluate_formulas=True, header=0).fillna("")
    if clips_df_raw.empty or len(clips_df_raw.columns) < len(AI_HEADER):
        clips_df = pd.DataFrame(columns=AI_HEADER)
    else:
        clips_df = clips_df_raw[AI_HEADER]

    return (tasks_data.copy(), clips_df.copy())

def list_task_choices(status_filter=None):
    """根據狀態篩選，生成下拉選單的選項(顯示標籤, 隱藏ID)"""
    global tasks_data
    if not tasks_data:
        return []

    filtered_tasks = tasks_data
    if status_filter:
        filtered_tasks = [t for t in tasks_data if t.get("status") == status_filter]

    # 顯示格式:[status] (P:priority) task — id
    def row_label(r):
        return f"[{r.get('status','N/A')}] (P:{r.get('priority','N/A')}) {r.get('task','Unknown')} — {r.get('id','N/A')}"

    return [(row_label(t), t.get("id")) for t in filtered_tasks if t.get("id")]

In [33]:
#執行初始化並載入資料到全域變數
tasks_data, clips_df = refresh_all()

def tasks_to_df(data):
    """將 List/Dicts 轉換成 Gradio Dataframe(Cn)"""

    #如果資料為空則建立一個空Dataframe
    if not data:
        df_display = pd.DataFrame(columns=list(TASKS_COL_MAP.keys()))
        return df_display

    #處理有資料
    df = pd.DataFrame(data)
    df_display = pd.DataFrame()

    for col_display, col_prog in TASKS_COL_MAP.items():
        if col_prog in df.columns:
            df_display[col_display] = df[col_prog]

    return df_display

def filter_tasks_df(filter_status):
    """根據Gradio Radio篩選Dataframe"""
    global tasks_data
    df = tasks_to_df(tasks_data)
    if filter_status == "Incomplete":
        df = df[df["狀態"] != "done"]
    elif filter_status == "Completed":
        df = df[df["狀態"] == "done"]
    return df

def get_todo_choices_and_df(tasks_data_list):
    """獲取未完成任務的下拉選單選項和Dataframe"""
    tasks_dataframe = tasks_to_df(tasks_data_list)
    new_todo_choices = list_task_choices(status_filter="todo")
    todo_df = tasks_dataframe[tasks_dataframe["狀態"] != "done"]
    return new_todo_choices, todo_df

def handle_add_task(task, p, e, d, l, n, pf):
    """新增任務"""
    global tasks_data
    _now = tznow().isoformat()
    new_task = {
        "id": str(uuid.uuid4())[:8],
        "task": task.strip(),
        "status": "todo",
        "priority": p or "M",
        "est_min": int(e) if e else 25,
        "actual_min": 0, "pomodoros": 0, "completed_period": "", "completed_date": "",
        "due_date": d or "", "labels": l or "", "notes": n or "",
        "created_at": _now, "updated_at": _now, "completed_at": "", "planned_for": pf or ""
    }
    tasks_data.append(new_task)
    write_data(ws_tasks, tasks_data, TASKS_COL_MAP)

    tasks_dataframe = filter_tasks_df("Incomplete") #預設結果
    new_choices = list_task_choices()
    new_todo_choices, todo_df = get_todo_choices_and_df(tasks_data)

    return "任務新增", tasks_dataframe, new_choices, new_todo_choices, todo_df

def handle_delete_task(task_id_input):
    """任務刪除"""
    global tasks_data

    if not task_id_input:
        return "請先選擇要刪除的任務", filter_tasks_df("Incomplete"), gr.update(), gr.update(), gr.update()

    new_tasks_data = [t for t in tasks_data if t.get("id") != task_id_input]

    if len(new_tasks_data) == len(tasks_data):
         return f"找不到 ID 為 {task_id_input} 的任務", filter_tasks_df("Incomplete"), gr.update(), gr.update(), gr.update()

    tasks_data = new_tasks_data
    write_data(ws_tasks, tasks_data, TASKS_COL_MAP)

    tasks_dataframe = filter_tasks_df("Incomplete") #預設結果
    new_choices = list_task_choices()
    new_todo_choices, todo_df = get_todo_choices_and_df(tasks_data)

    return f"任務 {task_id_input} 已刪除", tasks_dataframe, new_choices, new_todo_choices, todo_df

def complete_task_entry(task_id, actual_min, pomodoros, completed_period):
    """登記任務完成並將狀態設為done"""
    global tasks_data

    if not task_id:
        return "請選擇任務", filter_tasks_df("Incomplete"), gr.update(), gr.update()

    target_task = next((t for t in tasks_data if t.get("id") == task_id), None)

    if target_task is None:
        return "找不到任務", filter_tasks_df("Incomplete"), gr.update(), gr.update()

    target_task["status"] = "done"
    target_task["actual_min"] = int(actual_min) if actual_min else target_task["est_min"]
    target_task["pomodoros"] = int(pomodoros) if pomodoros else 1
    target_task["completed_period"] = completed_period
    target_task["completed_date"] = tznow().date().isoformat()
    target_task["completed_at"] = tznow().isoformat()
    target_task["updated_at"] = tznow().isoformat()

    write_data(ws_tasks, tasks_data, TASKS_COL_MAP)

    tasks_dataframe = filter_tasks_df("Incomplete") # 預設結果
    new_todo_choices, todo_df = get_todo_choices_and_df(tasks_data)

    # outputs:訊息,完整任務列表,未完成下拉選單,未完成Dataframe
    return f" 任務 {task_id} 已登記完成。", tasks_dataframe, new_todo_choices, todo_df

def handle_refresh():
    """處理重新整理:從Sheet讀取最新資料並更新介面元件"""

    tasks_data_list, clips_df_data = refresh_all()

    #Tasks分頁Dataframe
    tasks_df_display = filter_tasks_df("Incomplete")

    #刪除選項和完成選項
    new_choices = list_task_choices() #所有任務選擇(刪除)
    new_todo_choices = list_task_choices(status_filter="todo") #未完成任務選擇(完成登記)

    #Complete Task分頁Dataframe
    tasks_df_full = tasks_to_df(tasks_data_list)
    new_todo_df = tasks_df_full[tasks_df_full["狀態"] != "done"]

    #摘要
    new_summary = today_summary()

    msg = f"成功從Google Sheet重新整理資料。最後更新時間：{tznow().strftime('%H:%M:%S')}"

    #回傳順序必須與 outputs 一致：
    return (
        msg,             # 1. msg_app
        tasks_df_display, # 2. grid_tasks
        clips_df_data,    # 3. grid_clips
        new_choices,     # 4. task_choice_delete
        new_todo_choices, # 5. todo_task_choice
        new_todo_df,     # 6. grid_todo
        new_summary      # 7. out_summary
    )

In [34]:
import requests
import json
import pandas as pd
from dateutil.tz import gettz

#從環境變數中獲取API Key
API_KEY = os.environ.get("GEMINI_API_KEY")

BASE_GEMINI_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-09-2025:generateContent"

def tznow():
    """返回帶有時區的當前時間"""
    # 假設 TIMEZONE 在全局範圍內已定義(如 'Asia/Taipei')
    return dt.now(gettz(TIMEZONE))

def generate_today_plan():
    """使用 Gemini 根據今天的未完成任務生成行動計畫，並記錄到 ai_chat (使用 requests.post)"""
    global tasks_data, clips_df, ws_ai, AI_HEADER

    if not API_KEY:
        return "錯誤：Gemini API Key 尚未設置或無效，請檢查區塊 A 的 API Key 載入狀況。"

    #任務篩選與準備
    tasks_df_temp = pd.DataFrame(tasks_data)
    today = tznow().date().isoformat()
    #篩選今天到期或計畫於今天的未完成任務
    cand = tasks_df_temp[
        ((tasks_df_temp["due_date"]==today) | (tasks_df_temp["planned_for"].str.lower()=="today")) &
        (tasks_df_temp["status"]!="done")
    ].copy()

    if cand.empty:
        return "今天沒有標記的任務。請在 Tasks 分頁把任務的 due_date 設為今天或 planned_for 設為 today。"

    pr_order = {"High":0, "Mid":1, "Low":2}
    cand["p_ord"] = cand["priority"].map(pr_order).fillna(3)
    cand = cand.sort_values(["p_ord","est_min"], ascending=[True, True])

    items = []
    for _, r in cand.iterrows():
        est_min_val = int(r["est_min"]) if pd.notna(r["est_min"]) and r["est_min"] != "" else 0
        items.append({
            "id": r["id"], "task": r["task"], "est_min": est_min_val,
            "priority": r["priority"]
        })
    user_content_dict = {"today": today, "tasks": items}
    user_content = json.dumps(user_content_dict, ensure_ascii=False)

    #準備System Prompt
    sys_prompt = (
        "你是一位任務規劃助理。請把輸入的任務（含估時與優先級）排成三段：morning、afternoon、evening，"
        "並給出每段的重點、順序、每項的時間預估與備註。總時數請大致符合任務估時總和。"
        "回傳以 Markdown 條列，格式：\n"
        "### Morning\n- [任務ID] 任務名稱（預估 xx 分）— 備註\n...\n"
        "### Afternoon\n...\n### Evening\n...\n"
    )

    try:
        request_body = json.dumps({
            "contents": [{"parts": [{"text": user_content}]}],
            "config": {
                "systemInstruction": sys_prompt
            }
        })

        response = requests.post(
            f"{BASE_GEMINI_URL}?key={API_KEY}", # 正確的 URL 格式
            headers={'Content-Type': 'application/json'},
            data=request_body,
            timeout=60
        )

        response.raise_for_status()
        result = response.json()

        #解析Gemini
        plan_md = result.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', 'API 返回空結果。')

        if not plan_md or plan_md == 'API 返回空結果。':
            return f"Gemini 失敗：API 返回空結果，請檢查輸入或 API 狀態。\n原始響應:\n{json.dumps(result, indent=2, ensure_ascii=False)}"

        #寫入AI Chat Sheet
        new_row = {
            "時間": tznow().isoformat(),
            "任務列表": user_content,
            "AI建議": plan_md
        }
        new_log_df = pd.DataFrame([new_row])
        clips_df = pd.concat([clips_df, new_log_df[AI_HEADER]], ignore_index=True)
        write_data(ws_ai, clips_df.to_dict('records'), AI_HEADER)

        return plan_md

    except requests.exceptions.HTTPError as e:
        return f"Gemini API 呼叫失敗 (HTTP Error): {e}。請檢查 API Key 是否有效或是否有足夠的餘額。"
    except Exception as e:
        return f"Gemini 失敗：發生錯誤 ({type(e).__name__})：{e}\n\n請檢查 API Key 或網路連線。"


def today_summary():
    """計算今日計畫任務的完成率 (保持不變)"""
    global tasks_data

    today = tznow().date().isoformat()
    planned = [
        t for t in tasks_data
        if t.get("due_date") == today or t.get("planned_for", "").lower() == "today"
    ]
    done = [t for t in planned if t.get("status") == "done"]

    total = len(planned)
    done_n = len(done)
    rate = (done_n/total*100) if total>0 else 0
    return f"今日計畫任務：{total}；完成：{done_n}；完成率：{rate:.1f}%"

In [35]:
def _get_data_context(data_type):
    """根據類型獲取資料和表頭"""
    global tasks_data, clips_df, TASKS_HEADER_PROG, AI_HEADER, ws_tasks, ws_ai

    if data_type == 'tasks':
        df = pd.DataFrame(tasks_data)
        if df.empty:
            return None, TASKS_HEADER_PROG, ws_tasks, '任務'
        return df.to_dict('records'), TASKS_HEADER_PROG, ws_tasks, '任務'

    elif data_type == 'ai_chat':
        if clips_df.empty:
            return None, AI_HEADER, ws_ai, 'AI Chat'
        return clips_df.to_dict('records'), AI_HEADER, ws_ai, 'AI Chat'

    return None, None, None, None

def export_data(data_list, data_header, data_name, file_format="csv"):
    """將資料列表匯出為檔案"""
    if not data_list:
        return f"{data_name} 資料是空的，無法匯出。", None

    df = pd.DataFrame(data_list, columns=data_header)

    with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=f"_{data_name}.{file_format}") as tmp:
        file_path = tmp.name

        if file_format == "csv":
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
        elif file_format == "json":
            df.to_json(file_path, orient='records', indent=4, force_ascii=False)
        else:
            return f"不支援的檔案格式：{file_format}", None

    return f"{data_name} 資料已匯出為 {file_format.upper()} 檔案。", file_path

def export_wrapper(data_type, file_format):
    """匯出功能的 Gradio Wrapper"""
    data_list, data_header, _, data_name = _get_data_context(data_type)

    if data_list is None or not data_list:
        return f"{data_name} 資料目前是空的，無法匯出。請先確認 Google Sheet 中是否有資料。", None

    return export_data(data_list, data_header, data_name, file_format)

def import_data(file_obj, header_prog, ws_sheet, col_map=None):
    """核心匯入邏輯：讀取檔案，檢查欄位，並寫回 Google Sheet"""
    if file_obj is None:
        return "請先上傳檔案", None

    file_path = file_obj.name
    file_ext = os.path.splitext(file_path)[1].lower()

    try:
        if file_ext == '.csv':
            imported_df = pd.read_csv(file_path)
        elif file_ext == '.json':
            imported_df = pd.read_json(file_path)
        else:
            return "不支援的檔案格式，請上傳 .csv 或 .json 文件。", None

        missing_cols = [col for col in header_prog if col not in imported_df.columns]
        if missing_cols:
            return f"匯入失敗：缺少必要欄位 {', '.join(missing_cols)}", None

        imported_df = imported_df[header_prog].fillna("")
        new_data_list = imported_df.to_dict('records')
        write_data(ws_sheet, new_data_list, col_map)

        return f"資料已成功匯入 {len(new_data_list)} 筆，並同步至 Google Sheet。", new_data_list

    except Exception as e:
        return f"匯入發生錯誤：{e}", None

def handle_import_tasks(file_obj):
    """任務匯入的 Gradio Wrapper"""
    global tasks_data

    msg, new_data = import_data(file_obj, TASKS_HEADER_PROG, ws_tasks, TASKS_COL_MAP)

    if new_data is not None:
        tasks_data = new_data
        new_choices = list_task_choices()
        tasks_dataframe = filter_tasks_df("incomplete")
        new_todo_choices, todo_df = get_todo_choices_and_df(tasks_data)

        return msg, tasks_dataframe, new_choices, new_todo_choices, todo_df

    return msg, filter_tasks_df("incomplete"), list_task_choices(), list_task_choices(status_filter="todo"), get_todo_choices_and_df(tasks_data)[1]

def handle_import_ai_chat(file_obj):
    """AI Chat Log 匯入的 Gradio Wrapper"""
    global clips_df

    msg, new_data = import_data(file_obj, AI_HEADER, ws_ai, None)

    if new_data is not None:
        clips_df = pd.DataFrame(new_data, columns=AI_HEADER)
        return msg, clips_df

    return msg, clips_df

In [36]:
#Gradio

with gr.Blocks(title="待辦清單＋任務完成登記＋AI 計畫") as demo:
    gr.Markdown("# 待辦清單與任務完成登記（Google Sheet＋Gradio＋AI 計畫）")

    msg_app = gr.Markdown(value="App 已啟動。請點擊 '重新整理' 獲取最新資料。", visible=True)

    with gr.Row():
        btn_refresh = gr.Button("重新整理（Sheet → App）")
        out_summary = gr.Markdown(today_summary())

    with gr.Tab("Tasks"):
        gr.Markdown("## 任務管理")

        filter_radio = gr.Radio(["未完成", "全部", "已完成"], value="未完成", label="任務清單篩選")

        with gr.Row():
            with gr.Column(scale=2):
                task = gr.Textbox(label="任務名稱", placeholder="作業?休閒活動?開會???")
                priority = gr.Dropdown(["High","Mid","Low"], value="Mid", label="優先級")
                est_min = gr.Number(value=25, label="預估時間（分鐘）", precision=0)
                due_date = gr.Textbox(label="到期日（YYYY-MM-DD）")
                labels = gr.Textbox(label="標籤（逗號分隔，可空白）")
                notes = gr.Textbox(label="備註（可空白）")
                planned_for = gr.Dropdown(["","today","tomorrow"], value="", label="規劃歸屬")
                btn_add = gr.Button("➕ 新增任務")
                msg_add = gr.Markdown()
            with gr.Column(scale=3):
                grid_tasks = gr.Dataframe(value=filter_tasks_df("incomplete"), label="任務清單", interactive=False)

        gr.Markdown("---")
        gr.Markdown("### 刪除任務")
        with gr.Row():
            task_choice_delete = gr.Dropdown(
                choices=list_task_choices(),
                label="選取要刪除的任務",
                interactive=True, scale=3
            )
            delete_btn = gr.Button("刪除任務", variant="stop", scale=1)
            msg_delete = gr.Markdown()

    with gr.Tab("Complete Task"):
        gr.Markdown("## 任務完成登記")
        gr.Markdown("此頁面用於登記任務的實際完成資訊，並將狀態設為 **done**。")
        with gr.Row():
            tasks_df_temp = tasks_to_df(tasks_data)
            grid_todo = gr.Dataframe(
                value=tasks_df_temp[tasks_df_temp["狀態"] != "done"],
                label="未完成任務清單",
                interactive=False, scale=3
            )
            with gr.Column(scale=2):
                todo_task_choice = gr.Dropdown(
                    choices=list_task_choices(status_filter="todo"),
                    label="選擇完成的任務",
                    interactive=True
                )
                actual_min_input = gr.Number(value=25, label="實際花費時間 (分鐘)", precision=0)
                pomodoros_input = gr.Number(value=1, label="番茄鐘數", precision=0)
                period_input = gr.Radio(["morning", "afternoon", "evening"], label="完成時段", value="afternoon")
        with gr.Row():
            btn_complete = gr.Button("登記完成並更新任務清單", variant="primary")
            msg_complete = gr.Markdown()
    with gr.Tab("AI Plan"):
        gr.Markdown("## 產生今日計畫")
        gr.Markdown("把**今天的任務**排成 **morning / afternoon / evening** 三段行動計畫。結果會同步記錄到 AI Chat 分頁。")
        btn_plan = gr.Button("產生今日計畫並儲存紀錄", variant="primary")
        out_plan = gr.Markdown()

    with gr.Tab("AI Chat Log"):
        gr.Markdown("## AI 計畫紀錄")
        grid_clips = gr.Dataframe(value=clips_df, label="AI 計畫紀錄", interactive=False)

    with gr.Tab("Summary"):
        gr.Markdown("## 數據摘要")
        btn_summary = gr.Button("重新計算今日完成率")
        out_summary2 = gr.Markdown(value=today_summary())

    with gr.Tab("Data IO"):
        gr.Markdown("## 資料匯入/匯出管理")
        msg_io = gr.Markdown("選擇操作類型與檔案格式。")

        gr.Markdown("#### 匯出資料 (Download)")
        with gr.Row():
            data_type_export = gr.Radio(
                ["tasks", "ai_chat"],
                label="選擇匯出資料類型",
                value="tasks"
            )
            file_format_export = gr.Radio(
                ["csv", "json"],
                label="選擇檔案格式",
                value="csv"
            )
        with gr.Row():
            btn_export = gr.Button("點擊匯出")
            file_export = gr.File(label="下載檔案", file_count="single", interactive=False)

        gr.Markdown("---")
        gr.Markdown("#### 匯入資料 (Upload / **注意：將覆蓋 Google Sheet 紀錄！**)")

        gr.Markdown("##### 匯入任務紀錄 (Tasks)")
        with gr.Row():
            file_import_tasks = gr.File(label="上傳任務 CSV/JSON 檔案", file_count="single", file_types=['.csv', '.json'])
            btn_import_tasks = gr.Button("匯入任務(覆蓋)")

        gr.Markdown("##### 匯入 AI Chat Log 紀錄")
        with gr.Row():
            file_import_ai_chat = gr.File(label="上傳AI Chat Log CSV/JSON檔案", file_count="single", file_types=['.csv', '.json'])
            btn_import_ai_chat = gr.Button("匯入AI Log(覆蓋)")

    #綁定任務篩選器:選取後更新grid_tasks
    filter_radio.change(filter_tasks_df, inputs=filter_radio, outputs=grid_tasks)


    #重新整理按鈕
    btn_refresh.click(
        handle_refresh,
        inputs=None,
        outputs=[msg_app, grid_tasks, grid_clips, task_choice_delete, todo_task_choice, grid_todo, out_summary]
    )

    #任務管理
    btn_add.click(
        handle_add_task,
        inputs=[task, priority, est_min, due_date, labels, notes, planned_for],
        outputs=[msg_add, grid_tasks, task_choice_delete, todo_task_choice, grid_todo]
    )

    delete_btn.click(
        fn=handle_delete_task,
        inputs=[task_choice_delete],
        outputs=[msg_delete, grid_tasks, task_choice_delete, todo_task_choice, grid_todo]
    )

    #任務完成登記區
    btn_complete.click(
        complete_task_entry,
        inputs=[todo_task_choice, actual_min_input, pomodoros_input, period_input],
        outputs=[msg_complete, grid_tasks, todo_task_choice, grid_todo]
    )

    #AI Plan
    btn_plan.click(
        generate_today_plan,
        inputs=None,
        outputs=out_plan
    )

    #摘要
    btn_summary.click(today_summary, outputs=[out_summary2])

    #匯出
    btn_export.click(
        export_wrapper,
        inputs=[data_type_export, file_format_export],
        outputs=[msg_io, file_export]
    )

    #任務匯入
    btn_import_tasks.click(
        handle_import_tasks,
        inputs=[file_import_tasks],
        outputs=[msg_io, grid_tasks, task_choice_delete, todo_task_choice, grid_todo]
    )

    #AI Chat
    btn_import_ai_chat.click(
        handle_import_ai_chat,
        inputs=[file_import_ai_chat],
        outputs=[msg_io, grid_clips]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://968a6d9c1636552ae1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
